# Stock Market Question Answering Agent

- answer questions about individual stocks using (mostly) OpenBB functions
- uses the OpenAPI tools functionality
- define a set of functions
- when prompting, give OpenAI the set of functions and their descriptions in a specified format
- if OpenAI needs to run a function to respond to the prompt, it will provide the function signature and ask you to provide the return values before continuing
  

In [2]:
import os
import dotenv
import warnings
from datetime import datetime, timedelta

import yaml

import pandas as pd

import IPython
from IPython.display import HTML, Image, Markdown, display

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.memory import ChatMessageHistory
from langchain.tools import StructuredTool

import openbb
from openbb import obb
from openbb_core.app.model.obbject import OBBject

import openai
from openai import OpenAI
import tiktoken

import langchain
from langchain_core.tools import StructuredTool
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# langchain is still on pydantic v1
import pydantic.v1
from pydantic.v1 import BaseModel, Field # <-- Uses v1 namespace

import wikipedia

# paid API for edgar filings
import sec_api
from sec_api import QueryApi, ExtractorApi

# free API for edgar filing
from sec_downloader import Downloader
import sec_parser as sp

dotenv.load_dotenv()

from BB_agent_tool import BB_agent_tool, get_response, agent_query, bb_agent_system_prompt

# turn off excessive warnings
warnings.filterwarnings('ignore')


In [3]:
print(f'pandas         {pd.__version__}')
print(f'obb            {obb.system.version}')
print(f'openai         {openai.__version__}')
print(f'langchain      {langchain.__version__}')
print(f'pydantic       {pydantic.v1.__version__}')
print(f'wikipedia      {wikipedia.__version__}')


pandas         2.2.2
obb            4.2.1
openai         1.30.1
langchain      0.2.0
pydantic       1.10.15
wikipedia      (1, 4, 0)


# Connect to OpenBB

In [4]:
# login with email and password
obb.account.login(email=os.environ['OPENBB_USER'], password=os.environ['OPENBB_PW'], remember_me=True)

In [5]:
obb.equity.search("Merck", provider="sec").to_df().head(3)


,symbol,name,cik
0,MRK,"Merck & Co., Inc.",310158


In [7]:
symbol = "MRK"
company = "Merck"


In [8]:
obj = obb.equity.price.quote(symbol)
obj


OBBject

id: 0664e4d2-9af8-76f4-8000-379b8ab89417
results: [{'symbol': 'MRK', 'asset_type': None, 'name': 'Merck & Co., Inc.', 'excha...
provider: fmp
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'fmp'}, 'standa...

In [9]:
obj.results[0].model_dump()


{'symbol': 'MRK',
 'asset_type': None,
 'name': 'Merck & Co., Inc.',
 'exchange': 'NYSE',
 'bid': None,
 'bid_size': None,
 'bid_exchange': None,
 'ask': None,
 'ask_size': None,
 'ask_exchange': None,
 'quote_conditions': None,
 'quote_indicators': None,
 'sales_conditions': None,
 'sequence_number': None,
 'market_center': None,
 'participant_timestamp': None,
 'trf_timestamp': None,
 'sip_timestamp': None,
 'last_price': 131.07,
 'last_tick': None,
 'last_size': None,
 'last_timestamp': datetime.datetime(2024, 5, 22, 19, 53, 12, tzinfo=datetime.timezone.utc),
 'open': 130.17,
 'high': 131.6,
 'low': 130.15,
 'close': None,
 'volume': 2500185,
 'exchange_volume': None,
 'prev_close': 130.76,
 'change': 0.31,
 'change_percent': 0.002371,
 'year_high': 133.1,
 'year_low': 99.14,
 'price_avg50': 127.4142,
 'price_avg200': 115.0729,
 'avg_volume': 8234479,
 'market_cap': 331975406700.0,
 'shares_outstanding': 2532810000,
 'eps': 0.9,
 'pe': 145.63,
 'earnings_announcement': datetime.date

# Prompt OpenAI 

In [10]:
MODEL = "gpt-4o"

# MAX_INPUT_TOKENS = 65536
MAX_OUTPUT_TOKENS = 4096    # max in current model
MAX_RETRIES = 3
TEMPERATURE = 0


In [11]:
chat = ChatOpenAI(model=MODEL, temperature=0.0, streaming=True)

prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                bb_agent_system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

chain = prompt_template | chat

memory = ChatMessageHistory(llm=chat, max_token_limit=1000)

memory.add_user_message("What is the airspeed velocity of an unladen swallow")

text_response = ""
display_handle = display(HTML("<p>"), display_id=True)

for chunk in chain.stream({"messages": memory.messages}):
    text_response += chunk.content
    display_handle.update(HTML(f"<p>{text_response}</p>"))
    if hasattr(chunk, 'response_metadata'):
        response_metadata = chunk.response_metadata


# Question Answering Agent
### Create tools

In [12]:
# dict to store tools
tool_dict={}


In [13]:
class SymbolLimitSchema(BaseModel):
    symbol: str
    limit: int

class SymbolSchema(BaseModel):
    symbol: str

class QuerySchema(BaseModel):
    query: str = Field(description="The search string to match to the stock symbol.")
    # limit not supported for provider=sec
    # limit: int = Field(description="The maximum number of values to return")

schema_dict = {
    "SymbolSchema": SymbolSchema,
    "QuerySchema": QuerySchema,
    "SymbolLimitSchema": SymbolLimitSchema,
}


In [14]:
# make a custom tool

# metadata not used for custom tool
yaml_get_10k_item1_from_symbol="""
name: get_10k_item1_from_symbol
description: Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.
openapi_path: null
parameters:
  symbol:
    description: The symbol to get the 10-K item 1 for
    type: string
example_parameter_values:
- symbol: MSFT
"""
yaml.safe_load(yaml_get_10k_item1_from_symbol)

def fn_get_10k_item1_from_symbol(symbol):
    """
    Get item 1 of the latest 10-K annual report filing for a given symbol.

    Args:
        symbol (str): The symbol of the equity.

    Returns:
        str: The item 1 of the latest 10-K annual report filing, or None if not found.

    """
    item1_text = None
    try:
        # sec needs you to identify yourself for rate limiting
        dl = Downloader(os.getenv("SEC_FIRM"), os.getenv("SEC_USER"))
        html = dl.get_filing_html(ticker=symbol, form="10-K")
        elements: list = sp.Edgar10QParser().parse(html)
        tree = sp.TreeBuilder().build(elements)
        sections = [n for n in tree.nodes if n.text.startswith("Item")]
        item1_node = sections[0]
        item1_text = "\n".join([n.text for n in sections[0].get_descendants()])
    except:
        return None
    # always return a list of dicts
    return [{'item1': item1_text}]

get_10k_item1_from_symbol = StructuredTool.from_function(
    func=fn_get_10k_item1_from_symbol,
    name="get_10k_item1_from_symbol",
    description="Given a stock symbol, gets item 1 of the company's latest 10-K annual report filing.",
    args_schema=SymbolSchema
)

tool_dict["get_10k_item1_from_symbol"]= get_10k_item1_from_symbol
tool_response = get_10k_item1_from_symbol.run("MSFT")
len((tool_response)[0]['item1'])


143226

In [15]:
def obb_function_factory(fn_metadata):
    # get obb method object based on path, e.g. /api/v1/equity/price/quote -> obb.equity.price.quote
    parts = fn_metadata["openapi_path"].removeprefix("/api/v1/").split("/")
    op = obb
    for part in parts:
        op = op.__getattribute__(part)
    singular = fn_metadata.get("singular", 0)
    default_args = fn_metadata.get("default_parameters", {})
    override_args = fn_metadata.get("override_parameters", {})

    # return a closure based on value of op etc.
    def tool_fn(**kwargs):
        """call op and return results without obb metadata"""
        # always use override args
        for k, v in override_args.items():
            kwargs[k] = v
        # use default if not already present
        for k, v in default_args.items():
            if k not in kwargs:
                kwargs[k] = v
        retobj = op(**kwargs)
        retlist = [r.model_dump_json() for r in retobj.results]
        if len(retlist):
            if singular == 1:
                # return first
                return retlist[0]
            elif singular == -1:
                # return last
                return retlist[-1]
        # no retlist or not singular
        return retlist

    return tool_fn


In [16]:
# Load obb tool defs from YAML file
with open('obbtools.yaml', 'r') as file:
    fn_yaml_list = yaml.safe_load(file)

for fn_metadata in fn_yaml_list:

    # instantiate tool and add to tool_dict
    tool_dict[fn_metadata["name"]] = StructuredTool.from_function(
        func=obb_function_factory(fn_metadata),
        name=fn_metadata["name"],
        description=fn_metadata["description"],
        args_schema=schema_dict[fn_metadata["args_schema"]]
    )

    # test tool
    print(fn_metadata["name"])
    for p in fn_metadata['example_parameter_values']:
        print(tool_dict[fn_metadata["name"]].run(tool_input=dict(**p)))
    print()


get_equity_search_symbol
['{"symbol":"AVGO","name":"Broadcom Inc.","cik":"1730168"}']

get_equity_price_quote
['{"symbol":"AAPL","asset_type":"EQUITY","name":"Apple Inc.","exchange":"NMS","bid":190.45,"bid_size":100,"bid_exchange":null,"ask":190.54,"ask_size":200,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":190.49,"last_tick":null,"last_size":null,"last_timestamp":null,"open":192.25,"high":192.8231,"low":190.27,"close":null,"volume":25172509,"exchange_volume":null,"prev_close":192.35,"change":null,"change_percent":null,"year_high":199.62,"year_low":164.08,"ma_50d":174.9092,"ma_200d":180.8282,"volume_average":64331141.0,"volume_average_10d":52039090.0,"currency":"USD"}']

get_company_profile_json
['{"symbol":"NVDA","name":"NVIDIA Corporation","cik":"0001045810","cusip":"67066G104","isin":"US67066G1040","lei":null,"

### Create agent

In [20]:
prompt_template = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=bb_agent_system_prompt,)),
     MessagesPlaceholder(variable_name='chat_history', optional=True),
     HumanMessagePromptTemplate(prompt=PromptTemplate(
         input_variables=['input'], template='{input}')),
     MessagesPlaceholder(variable_name='agent_scratchpad')]
)

tool_list = list(tool_dict.values())
agent = create_tool_calling_agent(chat, tool_list, prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)


In [21]:
def escape_output(text):
    # escape $ characters to avoid markdown interpreting them as latex
    return text.replace('$', '\$', )


In [25]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)


response = agent_executor.invoke({"input": "get the latest price quote for Delta Air lines"})

display(Markdown(escape_output(response["output"])))



> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Delta Air Lines'}`


['{"symbol":"DAL","name":"DELTA AIR LINES, INC.","cik":"27904"}']
Invoking: `get_equity_price_quote` with `{'symbol': 'DAL'}`


['{"symbol":"DAL","asset_type":"EQUITY","name":"Delta Air Lines, Inc.","exchange":"NYQ","bid":51.51,"bid_size":800,"bid_exchange":null,"ask":51.52,"ask_size":800,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":51.3901,"last_tick":null,"last_size":null,"last_timestamp":null,"open":52.21,"high":52.3995,"low":51.14,"close":null,"volume":4418303,"exchange_volume":null,"prev_close":52.23,"change":null,"change_percent":null,"year_high":53.86,"year_low":30.6,"ma_50d":48.4066,"ma_200d":41.24935,"volume_average":9564903.0,"volume_average_10d":5408920.0,"currency":"USD"}']The latest pric

The latest price quote for Delta Air Lines, Inc. (symbol: DAL) is \$51.39. Here are some additional details:

- **Open:** \$52.21
- **High:** \$52.40
- **Low:** \$51.14
- **Previous Close:** \$52.23
- **52-Week High:** \$53.86
- **52-Week Low:** \$30.60
- **Average Volume (50 days):** 9,564,903
- **Average Volume (10 days):** 5,408,920

If you need more information, feel free to ask!

In [29]:
agent_executor = AgentExecutor(agent=agent, tools=tool_list, verbose=True)

with open("questions.txt") as infile:
    for str in infile:
        str = str.strip() 
        if str:
            print(str)
            if str[0] != '#': 
                response = agent_executor.invoke({"input": str})
                display(Markdown(escape_output(response["output"])))
                

Please summarize item 1 from the latest MSFT annual report


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'MSFT'}`


['{"symbol":"MSFT","name":"MICROSOFT CORP","cik":"789019"}']
Invoking: `get_10k_item1_from_symbol` with `{'symbol': 'MSFT'}`


[{'item1': "Note About Forward-Looking Statements\nThis report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are “forward-looking statements” within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: “Business” (Part I, Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K), and “Management’s Discussion and Analysis of Financial Condition and Results of Operations” (Part II, Item 7 of this Fo

### Summary of Item 1 from Microsoft's Latest 10-K Annual Report

**General Overview:**
- **Company Mission:** Microsoft aims to empower every person and organization on the planet to achieve more. The company focuses on creating local opportunities, growth, and impact globally.
- **Core Offerings:** Microsoft develops and supports software, services, devices, and solutions that deliver new value for customers. This includes cloud-based solutions, operating systems, productivity and collaboration applications, server applications, business solutions, and gaming products.

**Strategic Ambitions:**
1. **Reinvent Productivity and Business Processes:** 
   - Products like Office 365, Dynamics 365, and LinkedIn are central to this ambition.
   - Microsoft 365 integrates Office 365, Windows, and Enterprise Mobility + Security.
   - Microsoft Teams and Microsoft Viva enhance collaboration and employee experience.
   
2. **Build the Intelligent Cloud and Intelligent Edge Platform:**
   - Azure is a key component, offering hybrid consistency, developer productivity, AI capabilities, and trusted security.
   - Investments in AI, including a partnership with OpenAI, are crucial for transforming business operations.
   - Azure Arc and Azure AI are highlighted for governance, management, and AI solutions.

3. **Create More Personal Computing:**
   - Windows 11 and its features like Windows Copilot and Dev Home aim to enhance productivity.
   - Microsoft Edge and Bing, including Bing Chat, are designed to improve search and browsing experiences.
   - The Surface family and Xbox gaming platform are integral to this ambition.

**Corporate Social Responsibility:**
- **Sustainability:** Microsoft aims to be carbon negative, water positive, and zero waste by 2030. Progress includes reducing emissions, signing power purchase agreements, and protecting land.
- **Racial Equity Initiative:** Focuses on strengthening communities, engaging ecosystems, and increasing representation and inclusion.
- **Digital Skills:** Initiatives like Skills for Jobs and AI Skills Initiative aim to support job seekers and workers in developing necessary skills.

**Human Capital Resources:**
- **Workforce:** As of June 30, 2023, Microsoft employed approximately 221,000 people globally.
- **Culture:** Emphasizes growth mindset, diversity, and inclusion. Employee feedback is gathered through various surveys and channels.
- **Total Rewards and Pay Equity:** Microsoft monitors pay equity and career progress, offering competitive compensation and benefits.

**Operating Segments:**
1. **Productivity and Business Processes:** Includes Office, LinkedIn, and Dynamics.
2. **Intelligent Cloud:** Encompasses Azure, server products, and enterprise services.
3. **More Personal Computing:** Covers Windows, devices like Surface, gaming, and search and news advertising.

**Research and Development:**
- Microsoft invests in a wide range of technologies, including AI, cloud computing, and mixed reality. The company operates research facilities globally and collaborates with top universities.

**Distribution, Sales, and Marketing:**
- Products and services are distributed through OEMs, direct sales, and resellers. Microsoft also offers various licensing options to meet the needs of different organizations.

**Legal and Regulatory Risks:**
- Microsoft faces risks related to competition laws, anti-corruption regulations, data privacy, and intellectual property. The company is also subject to government scrutiny and litigation.

**Operational Risks:**
- Risks include cybersecurity threats, data privacy issues, and supply chain disruptions. Microsoft invests in security measures and infrastructure to mitigate these risks.

**Conclusion:**
Microsoft's strategic focus on AI, cloud computing, and personal computing, combined with its commitment to sustainability and social responsibility, positions the company for continued growth and innovation. However, it faces significant risks related to competition, regulation, and cybersecurity.

# note that airlines is not in the the desc for LUV
What is the stock symbol for Southwest airlines?


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Southwest Airlines'}`


['{"symbol":"LUV","name":"SOUTHWEST AIRLINES CO","cik":"92380"}']The stock symbol for Southwest Airlines is **LUV**.

> Finished chain.


The stock symbol for Southwest Airlines is **LUV**.

# note misspelling
What is item 1 from the 10k annual report for Microsfot?


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Microsoft'}`


['{"symbol":"MSFT","name":"MICROSOFT CORP","cik":"789019"}']
Invoking: `get_10k_item1_from_symbol` with `{'symbol': 'MSFT'}`


[{'item1': "Note About Forward-Looking Statements\nThis report includes estimates, projections, statements relating to our business plans, objectives, and expected operating results that are “forward-looking statements” within the meaning of the Private Securities Litigation Reform Act of 1995, Section 27A of the Securities Act of 1933, and Section 21E of the Securities Exchange Act of 1934. Forward-looking statements may appear throughout this report, including the following sections: “Business” (Part I, Item 1 of this Form 10-K), “Risk Factors” (Part I, Item 1A of this Form 10-K), and “Management’s Discussion and Analysis of Financial Condition and Results of Operations” (Part

...

As of the end of 2023, the trailing dividend yield for Merck & Co., Inc. (symbol: MRK) was approximately 2.72%.

what was the performance for Merck from 1 year ago?


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Merck'}`


['{"symbol":"MRK","name":"Merck & Co., Inc.","cik":"310158"}']
Invoking: `get_price_performance_json` with `{'symbol': 'MRK'}`


['{"symbol":"MRK","one_day":0.0026,"wtd":null,"one_week":0.0031371948999999997,"mtd":null,"one_month":0.0327714,"qtd":null,"three_month":0.0142349,"six_month":0.2893391,"ytd":0.1577181,"one_year":0.14507820000000002,"two_year":null,"three_year":0.7396497,"four_year":null,"five_year":0.6977467,"ten_year":1.4340884,"max":330.58813260000005}']The performance of Merck & Co., Inc. (symbol: MRK) over the past year is approximately 14.51%.

> Finished chain.


The performance of Merck & Co., Inc. (symbol: MRK) over the past year is approximately 14.51%.

which ETF has the highest weight in Merck?


> Entering new AgentExecutor chain...

Invoking: `get_equity_search_symbol` with `{'query': 'Merck'}`


['{"symbol":"MRK","name":"Merck & Co., Inc.","cik":"310158"}']
Invoking: `get_etf_equity_exposure_json` with `{'symbol': 'MRK'}`


['{"equity_symbol":"MRK","etf_symbol":"VTSAX","shares":79744771.0,"weight":0.0067,"market_value":10304619308.62}', '{"equity_symbol":"MRK","etf_symbol":"VTI","shares":79744771.0,"weight":0.0067,"market_value":10304619308.62}', '{"equity_symbol":"MRK","etf_symbol":"VSMPX","shares":79744771.0,"weight":0.0067,"market_value":10304619308.62}', '{"equity_symbol":"MRK","etf_symbol":"VITSX","shares":79744771.0,"weight":0.0067,"market_value":10304619308.62}', '{"equity_symbol":"MRK","etf_symbol":"VOO","shares":64468278.0,"weight":0.0077,"market_value":8330590883.16}', '{"equity_symbol":"MRK","etf_symbol":"VFINX","shares":64468278.0,"weight":0.0077,"market_value":8330590883.16}', '{"equity_symbol":"MRK","etf_symbol":"VFI

The ETF with the highest weight in Merck (symbol: MRK) is the **Health Care Select Sector SPDR Fund (XLV)**, with a weight of approximately 6.11%.